In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --user tqdm

In [3]:
from tqdm.notebook import tqdm

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import time
import random

def scrape_amazon_product_data(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    product_data = []
    
    # Find all product containers
    for product in soup.find_all('div', {'data-component-type': 's-search-result'}):
        product_name = product.h2.text.strip()
        try:
            product_price = product.find('span', 'a-price').find('span', 'a-offscreen').text
        except AttributeError:
            product_price = 'N/A'  # Handle missing price
        
        product_data.append({
            'product_name': product_name,
            'product_price': product_price
        })
    return product_data


In [50]:
# Scrape data from Amazon
product1=input("Enter the product you want recommendations of :- ")
base_url = 'https://www.amazon.in/s?k='+product1  # Example search for laptops
scraped_data = []
for i in tqdm(range(1, 10)):
    url = base_url.format(i)
    scraped_data.extend(scrape_amazon_product_data(url))

# Create a DataFrame from the scraped data
df = pd.DataFrame(scraped_data)
df.head()

Enter the product you want recommendations of :-  guitar


  0%|          | 0/9 [00:00<?, ?it/s]

,product_name,product_price
0,"Juârez Acoustic Guitar Kit, 38 Inch Cutaway, 0...","₹2,179"
1,"Intern INT-38C Right hand Acoustic Guitar Kit,...","₹2,179"
2,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","₹2,198"
3,Juârez JRZ38C/WH 6 Strings Acoustic Guitar 38 ...,"₹2,199"
4,Intern INT-38C-SB Right hand Acoustic Guitar K...,"₹2,198"


In [51]:
# Simulate user ratings for a realistic scenario
user_ids = [1, 2, 3, 4, 5]  # Example user IDs
num_ratings = 20  # Total number of ratings to simulate

user_ratings = []
for _ in range(num_ratings):
    user_id = random.choice(user_ids)  # Randomly select a user ID
    product_name = random.choice(df['product_name'].tolist())  # Randomly select a product name
    rating = random.randint(1, 5)  # Randomly assign a rating between 1 and 5
    user_ratings.append({'user_id': user_id, 'product_name': product_name, 'rating': rating})

user_ratings_df = pd.DataFrame(user_ratings)
print(user_ratings_df)

    user_id                                       product_name  rating
0         2  Intern INT-38C-NT-G 38 Incheses lindenwood Rig...       5
1         1  BLUEBERRY, B-38C Acoustic Guitar Kit with Bag,...       3
2         4  BLUEBERRY, B-38C Acoustic Guitar Kit with Bag,...       4
3         1  Kadence A281 Professional Acoustic Rosewood gu...       2
4         4  Yamaha FS80C 25 Incheses sprucewood The Ultima...       3
5         5          YAMAHA FS80C BLACK CONCERT CUTAWAY GUITAR       5
6         5  JUAREZ Engineered Wood Acoustic Guitar, 38 Inc...       3
7         4       Intern INT-38C Acoustic Guitar Kit (Natural)       5
8         1  Intern INT-38C-BK-G Cutaway Right Handed Acous...       5
9         1  Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...       4
10        2  YAMAHA Mexa Yamaha F310, Dreadnought Acoustic ...       1
11        4  Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...       5
12        1  Intern INT-38C-NT-G 38 Incheses lindenwood Rig...       5
13    

In [52]:
# Aggregate ratings by taking the mean of duplicates
user_ratings_agg = user_ratings_df.groupby(['user_id', 'product_name'])['rating'].mean().reset_index()

# Create a user-item matrix
user_item_matrix = user_ratings_agg.pivot(index='user_id', columns='product_name', values='rating').fillna(0)

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# The rest of your code remains the same
def get_recommendations(user_id, n=5):
    user_row = user_item_matrix.loc[user_id]
    similar_users = user_similarity[user_id - 1]  # Adjust for 0-based indexing
    
    # Get products the user hasn't rated
    unrated_products = user_row[user_row == 0].index
    
    # Calculate predicted ratings
    predicted_ratings = {}
    for product in unrated_products:
        product_ratings = user_item_matrix[product]
        weighted_sum = sum(similar_users * product_ratings)
        similarity_sum = sum(similar_users)
        predicted_ratings[product] = weighted_sum / similarity_sum if similarity_sum != 0 else 0
    
    # Sort and return top N recommendations
    sorted_recommendations = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations[:n]

# Example usage
user_id = 4
recommendations = get_recommendations(user_id)
print(f"Top 5 product recommendations for User {user_id}:")
for product, predicted_rating in recommendations:
    print(f"\n{product}: Predicted rating = {predicted_rating:.2f}")

Top 5 product recommendations for User 4:

Intern INT-38C-BK-G Cutaway Right Handed Acoustic Guitar Kit, with Bag, Nickel Strings, Pick And Strap (Black) -6 Strings: Predicted rating = 0.71

Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black (Acoustic Guitar Kit, BLUE SUNBURST): Predicted rating = 0.57

Yamaha F280 Acoustic Rosewood Guitar (Natural, Beige): Predicted rating = 0.33

Kadence A281 Professional Acoustic Rosewood guitar (Natural, Beige): Predicted rating = 0.28

Intern INT-38C Acoustic Guitar Kit (Red): Predicted rating = 0.20


In [53]:
df.to_csv('commerse_data.csv', index=False)
user_ratings_df.to_csv('user_ratings.csv' , index=False)